In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
from squad_load import get_glove_train_embs, get_squad_train_voc, load_squad_train
from itertools import chain

%aimport QANet
%aimport HRED_v2

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [48]:
squad_path = '/pio/data/data/squad/'
glove_path = '/pio/data/data/glove_vec/6B/'

data = load_squad_train(squad_path)
i_to_w, w_to_i, voc_size = get_squad_train_voc(squad_path)
glove_embs = get_glove_train_embs(squad_path, glove_path)

def get_w(idx):
    return i_to_w[idx]

def filter_broken_answers(data):
    return [d for d in data if d[0]]

print 'Examples total:', len(data)

data = filter_broken_answers(data)

print 'Working examples:', len(data)

Examples total: 87599
Working examples: 86474


In [3]:
def trim_data(data, trim=100):
    return [d for d in data if max(map(len, d[1])) <= trim]

data_trimmed = trim_data(data)
len(data), len(data_trimmed)

(87599, 86789)

## QANet

In [49]:
qa_net = QANet.QANet(voc_size=voc_size,
                     emb_size=300,
                     rec_size=300)

Building the model...
Compiling theano functions...
Done


In [50]:
qa_net.train_one_epoch(data, 5, log_interval=20) # glove init

Done 20 batches in 7.42s	training loss:	10.478611
Done 40 batches in 14.69s	training loss:	10.027485
Done 60 batches in 22.49s	training loss:	9.594314
Done 80 batches in 29.89s	training loss:	9.252035
Done 100 batches in 37.41s	training loss:	8.876036
Done 120 batches in 44.87s	training loss:	8.926693
Done 140 batches in 51.85s	training loss:	8.656413
Done 160 batches in 58.01s	training loss:	8.347217
Done 180 batches in 64.93s	training loss:	8.118872
Done 200 batches in 71.03s	training loss:	7.919667
Done 220 batches in 78.75s	training loss:	7.795685
Done 240 batches in 85.78s	training loss:	7.673677
Done 260 batches in 92.36s	training loss:	7.615648
Done 280 batches in 99.02s	training loss:	7.540717
Done 300 batches in 105.88s	training loss:	7.519067
Done 320 batches in 111.58s	training loss:	7.349733
Done 340 batches in 118.06s	training loss:	7.233174
Done 360 batches in 126.22s	training loss:	7.179774
Done 380 batches in 131.95s	training loss:	7.092907
Done 400 batches in 136.90s	t

3.564743398702102

## HRED v2

In [4]:
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300,
                lv2_rec_size=300,
                out_emb_size=300,
                emb_init=glove_embs)

Building the model...
Compiling theano functions...
Done


In [5]:
ans = hred_net.get_output(data[:12], 3)
ans[0]

[array([ 0.6484955 ,  0.39728042,  0.29447019,  0.21638568,  0.34087649,
         0.42938796,  0.48248306,  0.45450044,  0.63264418], dtype=float32),
 array([ 0.43808964,  0.31143361,  0.32550314,  0.41264656,  0.40189919,
         0.67630386,  0.55489218,  0.2793248 ,  0.29060629,  0.52473378,
         0.40135068,  0.42928901,  0.24922398,  0.29269892,  0.26922536,
         0.37188664], dtype=float32),
 array([ 0.48105857,  0.35441753,  0.33231139,  0.33165267,  0.25691301,
         0.29037777,  0.38888532,  0.45136821,  0.26165053,  0.35437146,
         0.36461809,  0.29422539,  0.29016727,  0.75232959,  0.50326639,
         0.4879126 ,  0.19824371,  0.41686958,  0.44867724,  0.63904244,
         0.59564072,  0.3762525 ,  0.55565   ,  0.20368919,  0.45814279,
         0.13423204,  0.25729677,  0.28506696,  0.130806  ,  0.43678874], dtype=float32),
 array([ 0.36110905,  0.18258747,  0.213524  ,  0.28226432,  0.41397128,
         0.32674885,  0.29788589,  0.61358768,  0.34168947,  0.22

In [12]:
list(chain(*ans[8]))[46]

0.05232447

In [11]:
get_w(list(chain(*data[8][1][1:]))[45])

u'september'

In [10]:
list(reversed(np.array(list(chain(*ans[8]))).argsort()))

[46,
 23,
 45,
 74,
 7,
 19,
 49,
 8,
 6,
 126,
 10,
 24,
 28,
 80,
 9,
 29,
 54,
 43,
 31,
 37,
 4,
 42,
 123,
 82,
 11,
 30,
 111,
 94,
 16,
 35,
 81,
 15,
 50,
 148,
 62,
 78,
 53,
 91,
 204,
 33,
 13,
 12,
 105,
 21,
 20,
 155,
 152,
 122,
 36,
 158,
 125,
 203,
 217,
 5,
 22,
 84,
 14,
 87,
 27,
 34,
 3,
 85,
 107,
 222,
 186,
 79,
 25,
 110,
 128,
 56,
 175,
 60,
 127,
 131,
 52,
 117,
 63,
 149,
 112,
 47,
 26,
 66,
 55,
 2,
 32,
 86,
 197,
 171,
 151,
 61,
 108,
 83,
 137,
 119,
 124,
 48,
 67,
 166,
 201,
 134,
 239,
 1,
 172,
 44,
 236,
 90,
 233,
 106,
 77,
 240,
 104,
 176,
 71,
 141,
 75,
 0,
 227,
 142,
 101,
 109,
 198,
 147,
 202,
 154,
 73,
 207,
 18,
 235,
 226,
 93,
 65,
 59,
 150,
 120,
 194,
 113,
 195,
 205,
 121,
 118,
 153,
 57,
 232,
 39,
 17,
 216,
 132,
 41,
 140,
 247,
 116,
 178,
 64,
 97,
 179,
 100,
 58,
 224,
 76,
 170,
 70,
 69,
 51,
 243,
 103,
 133,
 169,
 136,
 218,
 237,
 238,
 143,
 72,
 174,
 167,
 102,
 225,
 177,
 200,
 89,
 181,
 114,
 146,
 20

In [ ]:
# Zwykły trening nie robił nic: wartości zwracane przez sieć dla zbioru uczącego nie miały żadnego sensu.
# Największe prawdopodobieństwa były prawie niezależne od pytania, miało ono mały wpływ na to, co się dzieje.
# Dla kosztu weighted_bin_ce mocno skrzywionego w stronę sytuacji t=1 wyszło podobnie.
#
# Następny krok: użyć zanurzeń GloVe, dodać cechy z https://arxiv.org/abs/1703.04816
#
# UPDATE:
# Obie cechy podobieństwa kontekstu do pytania dodane.
# W glove.6B jest około 70% słów ze zbioru uczącego SQuAD.
# W tej chwili te wektory biorę z glove.6B.300d, a pozostałym daję losowy init, 
# po czym wszystkie zanurzenia są dalej uczone.

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # weighted cost, random init

Done 20 batches in 7.96s	training loss:	0.813393
Done 40 batches in 16.89s	training loss:	0.618499
Done 60 batches in 25.51s	training loss:	0.537964
Done 80 batches in 33.72s	training loss:	0.476587
Done 100 batches in 41.79s	training loss:	0.436442
Done 120 batches in 48.56s	training loss:	0.419979
Done 140 batches in 57.10s	training loss:	0.409618
Done 160 batches in 65.07s	training loss:	0.408088
Done 180 batches in 74.05s	training loss:	0.397444
Done 200 batches in 81.68s	training loss:	0.401948
Done 220 batches in 90.27s	training loss:	0.401396
Done 240 batches in 98.95s	training loss:	0.410639
Done 260 batches in 107.43s	training loss:	0.418680
Done 280 batches in 115.18s	training loss:	0.427639
Done 300 batches in 123.33s	training loss:	0.426674
Done 320 batches in 131.00s	training loss:	0.422292
Done 340 batches in 139.84s	training loss:	0.424456
Done 360 batches in 148.91s	training loss:	0.415105
Done 380 batches in 156.67s	training loss:	0.416838
Done 400 batches in 163.90s	t

KeyboardInterrupt: 

In [5]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with random init

Done 20 batches in 2.25s	training loss:	0.228349
Done 40 batches in 5.03s	training loss:	0.154742
Done 60 batches in 7.65s	training loss:	0.126147
Done 80 batches in 10.05s	training loss:	0.106807
Done 100 batches in 12.52s	training loss:	0.095184
Done 120 batches in 14.64s	training loss:	0.090502
Done 140 batches in 17.25s	training loss:	0.087078
Done 160 batches in 19.51s	training loss:	0.085557
Done 180 batches in 22.31s	training loss:	0.082334
Done 200 batches in 24.40s	training loss:	0.082256
Done 220 batches in 26.96s	training loss:	0.081215
Done 240 batches in 29.60s	training loss:	0.082376
Done 260 batches in 32.51s	training loss:	0.083784
Done 280 batches in 34.97s	training loss:	0.084956
Done 300 batches in 37.36s	training loss:	0.084330
Done 320 batches in 39.52s	training loss:	0.083391
Done 340 batches in 42.24s	training loss:	0.083563
Done 360 batches in 44.99s	training loss:	0.081207
Done 380 batches in 47.14s	training loss:	0.081512
Done 400 batches in 49.09s	training lo

KeyboardInterrupt: 

In [6]:
hred_net.train_one_epoch(data_trimmed, 5, log_interval=20) # normal binary cross-entropy with glove init

Done 20 batches in 2.66s	training loss:	0.247073
Done 40 batches in 5.97s	training loss:	0.187469
Done 60 batches in 9.10s	training loss:	0.148785
Done 80 batches in 11.96s	training loss:	0.123427
Done 100 batches in 14.91s	training loss:	0.108109
Done 120 batches in 17.43s	training loss:	0.102956
Done 140 batches in 20.55s	training loss:	0.097911
Done 160 batches in 23.25s	training loss:	0.095246
Done 180 batches in 26.59s	training loss:	0.090887
Done 200 batches in 30.07s	training loss:	0.090136
Done 220 batches in 33.13s	training loss:	0.088647
Done 240 batches in 36.28s	training loss:	0.089258
Done 260 batches in 39.74s	training loss:	0.091964
Done 280 batches in 42.68s	training loss:	0.093008
Done 300 batches in 45.52s	training loss:	0.091937
Done 320 batches in 48.11s	training loss:	0.090498
Done 340 batches in 51.34s	training loss:	0.090224
Done 360 batches in 54.62s	training loss:	0.087337
Done 380 batches in 57.19s	training loss:	0.087323
Done 400 batches in 59.53s	training lo

KeyboardInterrupt: 

In [ ]:
hred_net.save_params()